In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

In [2]:
! pip install --upgrade langchain
! pip install --upgrade openai==0.27.8
! pip install -U langchain-openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/75/d1/06a969e3b15429873d52cb2ca8ad9b2c3b28bcf977ba6796dbaef13f5b95/openai-1.14.0-py3-none-any.whl (257 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.27.8
    Uninstalling openai-0.27.8:
      Successfully uninstalled openai-0.27.8
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.0.5
    Uninstalling langchain-openai-0.0.5:
      Successfully uninstalled langchain-openai-0.0.5
ERROR: pip's dependency resolver does not currently take into account all the packa

In [3]:
! pip show openai
! pip show langchain
! pip show langchain-openai

Name: openai
Version: 1.14.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: doctran, langchain-openai
Name: langchain
Version: 0.1.12
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental
Name: langchain-openai
Version: 0.0.8
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/lan

## Embed_documents
<hr>

In [4]:
from langchain_openai import OpenAIEmbeddings

e_model = OpenAIEmbeddings()
ebeddings = e_model.embed_documents(
     [
        "你好",
        "你好啊",
        "你叫什么名字?",
        "我叫王大锤",
        "很高兴认识你大锤",
    ]
)
ebeddings

[[0.0003300555516486924,
  -0.0060982073022533625,
  -0.002391764280324817,
  -0.029246050037373125,
  -0.04374236023339329,
  0.013926087440424438,
  -0.022441402872292373,
  -0.008470963986649177,
  -0.015218590925080091,
  -0.019387547613221756,
  0.03550582132294638,
  0.0014572338484260309,
  0.004808871711447434,
  -0.002569166801570725,
  -0.008337912444960724,
  -0.01678986906851155,
  0.03411194430377691,
  -0.01516790462348449,
  0.0182597736774292,
  -0.020008454807767872,
  -4.872502076702215e-07,
  0.0012521123724103544,
  0.009281946674824012,
  0.0008537489641078078,
  -0.007767690155026299,
  -0.007976771614769458,
  0.009985220040785584,
  -0.018335803129822602,
  0.007165789392255928,
  -0.008496307137446977,
  0.013165791053845206,
  0.01077719536586207,
  -0.02980360121756995,
  -0.002849525872932698,
  0.012000004254501163,
  -0.01348258137014032,
  -0.012291451419998477,
  -0.0018753966514970222,
  0.020870123797538307,
  0.005017953171190593,
  0.0170052858502928

## embed_query
<hr>

In [5]:
embedded_query = e_model.embed_query("这段对话中提到了什么名字?")
embedded_query[:5]

[0.004047733608954853,
 0.0009470974041103827,
 0.029676812733286713,
 -0.00633586830824844,
 -0.02480508870743322]

## 嵌入向量缓存
<hr>

In [8]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import  LocalFileStore
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
u_embeddings = OpenAIEmbeddings()
fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    u_embeddings,
    fs,
    namespace=u_embeddings.model,
)
list(fs.yield_keys())

['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
 'text-embedding-ada-002f05b40fb-a095-546e-9c5d-49e069720828',
 'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
 'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
 'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
 'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']

In [9]:
#加载文档，切分文档，将切分文档向量化病存储在缓存中

raw_documents = TextLoader("letter.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=600,chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 610, which is longer than the specified 600


In [ ]:
! pip install faiss-cup

In [10]:
from langchain.vectorstores import  FAISS
%timeit -r  1 -n 1 db= FAISS.from_documents(documents,cached_embeddings)

74.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
#查看缓存中的键
list(fs.yield_keys())

['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
 'text-embedding-ada-002f05b40fb-a095-546e-9c5d-49e069720828',
 'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
 'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
 'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
 'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']